Startup stuff, sourced from `startup.ipy`

In [1]:
# %load startup.ipy
#! /usr/bin/env python3

import sys
sys.path.append('./python')

import logging.config
import os

import xbx.database as xbxdb
import xbx.util as xbxu
import xbx.config as xbxc
import xbx.build as xbxb
import xbx.run as xbxr

logging.config.fileConfig("logging.ini", disable_existing_loggers=False)

CONFIG_PATH="config.ini"

xbxdb.init(xbxu.get_db_path(CONFIG_PATH))
config = xbxc.Config(CONFIG_PATH)

dbsession = xbxdb.scoped_session()


List RunSessions, ordered by descending timestamp

In [2]:
s=dbsession
l = s.query(xbxr.RunSession).order_by(xbxr.RunSession.timestamp.desc())
print([i.timestamp for i in l])

[datetime.datetime(2015, 4, 23, 18, 2, 52, 424848), datetime.datetime(2015, 4, 23, 16, 23, 9, 976050)]


Print latest RunSession

In [3]:
rs=l.first()
print(rs)

({'timestamp': datetime.datetime(2015, 4, 23, 18, 2, 52, 424848), 'xbh_mac': '001AB602D065', 'xbh_rev': 'e57b1b6', 'xbx_version': '4e5a30f94bcdb8670ae6d1a0fbb0aa1b54389142', 'host': 'zeus', 'build_session_id': 5, 'config_hash': 'eec557a96a561de008988fa1da87cca2e009340a69d9e1fbef49d0033276bc93', 'id': 2},['drift_measurements', 'build_execs', 'build_session', 'config'])


We have overridden the `__repr__` function in the base class for SqlAlchemy tables to print out the type, a dictionary of contents, and a list of relations.

Let's print out all build executions:

In [4]:
print([i for i in rs.build_execs])

[({'build_id': 16, 'test_ok': True, 'run_session_id': 2, 'id': 2},['build', 'runs', 'run_session']), ({'build_id': 17, 'test_ok': False, 'run_session_id': 2, 'id': 4},['build', 'runs', 'run_session']), ({'build_id': 18, 'test_ok': False, 'run_session_id': 2, 'id': 3},['build', 'runs', 'run_session']), ({'build_id': 19, 'test_ok': True, 'run_session_id': 2, 'id': 5},['build', 'runs', 'run_session'])]


Not much information here. Let's print out information on the builds associated with the build executions:

In [5]:
print([(i, i.build) for i in rs.build_execs])

[(({'build_id': 16, 'test_ok': True, 'run_session_id': 2, 'id': 2},['build', 'runs', 'run_session']), ({'timestamp': datetime.datetime(2015, 4, 23, 17, 53, 48, 579364), 'compiler_idx': 0, 'primitive_name': '0cipher', 'data': 0, 'log': b'LINK\nPOSTLINK\n', 'operation_name': 'crypto_aead', 'id': 16, 'hex_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.hex', 'build_ok': True, 'hex_checksum': '543c6c94056482ac551f46579b6c185a2ee2e3b33fe8d08af334e71c84f44a12', 'work_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0', 'platform_hash': '4315375386478985adb47e513968dc493b5731b2b8c24c1f91ef6abbc146b4e7', 'exe_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.bin', 'text': 14380, 'bss': 5320, 'implementation_hash': '1709ac0c04e3657b91bce5f4c570627b89fc1cfd6c943f950b8cade256ef05ec', 'build_session_id': 5, 'rebuilt': True, 'parallel_make': False},['platform', 'compiler', 'operation', 'primitive', 'implementation', 'build_session'])), (({'build_id': 17, 'test_ok'

Not very readable. Let's use prettyprint:

In [6]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint([(i, i.build) for i in rs.build_execs])

[   (   ({'build_id': 16, 'test_ok': True, 'run_session_id': 2, 'id': 2},['build', 'runs', 'run_session']),
        ({'timestamp': datetime.datetime(2015, 4, 23, 17, 53, 48, 579364), 'compiler_idx': 0, 'primitive_name': '0cipher', 'data': 0, 'log': b'LINK\nPOSTLINK\n', 'operation_name': 'crypto_aead', 'id': 16, 'hex_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.hex', 'build_ok': True, 'hex_checksum': '543c6c94056482ac551f46579b6c185a2ee2e3b33fe8d08af334e71c84f44a12', 'work_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0', 'platform_hash': '4315375386478985adb47e513968dc493b5731b2b8c24c1f91ef6abbc146b4e7', 'exe_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.bin', 'text': 14380, 'bss': 5320, 'implementation_hash': '1709ac0c04e3657b91bce5f4c570627b89fc1cfd6c943f950b8cade256ef05ec', 'build_session_id': 5, 'rebuilt': True, 'parallel_make': False},['platform', 'compiler', 'operation', 'primitive', 'implementation', 'build_session'])),
    (   ({'bui

Better, but not good. The overridden repr implementation is supposed to be `eval`able. Note that we need to import datetime and call repr explicitly. Let's try:

In [9]:
import datetime
pp.pprint([(eval(repr(i)), eval(repr(i.build))) for i in rs.build_execs])

[   (   (   {'build_id': 16, 'id': 2, 'run_session_id': 2, 'test_ok': True},
            ['build', 'runs', 'run_session']),
        (   {   'bss': 5320,
                'build_ok': True,
                'build_session_id': 5,
                'compiler_idx': 0,
                'data': 0,
                'exe_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.bin',
                'hex_checksum': '543c6c94056482ac551f46579b6c185a2ee2e3b33fe8d08af334e71c84f44a12',
                'hex_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.hex',
                'id': 16,
                'implementation_hash': '1709ac0c04e3657b91bce5f4c570627b89fc1cfd6c943f950b8cade256ef05ec',
                'log': b'LINK\nPOSTLINK\n',
                'operation_name': 'crypto_aead',
                'parallel_make': False,
                'platform_hash': '4315375386478985adb47e513968dc493b5731b2b8c24c1f91ef6abbc146b4e7',
                'primitive_name': '0cipher',
                'r

Much better. We can see the 0hash and icepole implementations succeeded but the 0hash implementation failed. The log is mostly empty since we've rebuilt and thus there's not much makefile output. 